In [8]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Load match data from both excel files
df_2023 = pd.read_excel('2023match_data.xlsx', sheet_name='Match Stats')
df_2024 = pd.read_excel('2024match_data.xlsx', sheet_name='Match Stats')

# Add season identifier to distinguish between datasets
df_2023['Season'] = 2023
df_2024['Season'] = 2024

# Ensure date columns are properly formatted
if 'Date' in df_2023.columns:
    df_2023['Date'] = pd.to_datetime(df_2023['Date'], errors='coerce')
else:
    # Create placeholder dates if not present
    df_2023['Date'] = pd.date_range(start='2023-01-01', periods=len(df_2023), freq='D')

if 'Date' in df_2024.columns:
    df_2024['Date'] = pd.to_datetime(df_2024['Date'], errors='coerce')
else:
    # Create placeholder dates if not present, starting after 2023 dates
    df_2024['Date'] = pd.date_range(start='2024-01-01', periods=len(df_2024), freq='D')

# Check for column consistency between datasets
missing_cols_2023 = set(df_2024.columns) - set(df_2023.columns)
missing_cols_2024 = set(df_2023.columns) - set(df_2024.columns)

# Add missing columns with NaN values to ensure consistency
for col in missing_cols_2023:
    if col != 'Season':  # Skip the Season column we added
        df_2023[col] = np.nan

for col in missing_cols_2024:
    if col != 'Season':  # Skip the Season column we added
        df_2024[col] = np.nan

# Combine the datasets
df = pd.concat([df_2023, df_2024], ignore_index=True)

# Filter out any matches involving team '0'
df = df[(df['home_team'] != '0') & (df['away_team'] != '0')]
print("Total matches after filtering team '0':", len(df))

# Standardize team names (ensure consistency between years)
# Define mapping of any inconsistent team names
# Define mapping of inconsistent team names - update after loading datasets but before combining them
team_name_mapping = {
    'Boston U.': 'BU',
    'Boston University': 'BU',
    'Loyola Maryland': 'Loyola',
}

# Apply team name standardization to both datasets
df_2023['home_team'] = df_2023['home_team'].replace(team_name_mapping)
df_2023['away_team'] = df_2023['away_team'].replace(team_name_mapping)
df_2024['home_team'] = df_2024['home_team'].replace(team_name_mapping)
df_2024['away_team'] = df_2024['away_team'].replace(team_name_mapping)

# Combine the datasets ONLY ONCE
df = pd.concat([df_2023, df_2024], ignore_index=True)

# Print unique teams to verify standardization
print("Teams after standardization:", sorted(list(set(df['home_team'].unique()) | set(df['away_team'].unique()))))
print("Total unique teams after standardization:", len(set(df['home_team'].unique()) | set(df['away_team'].unique())))

# REMOVE these lines - don't standardize again
# df['home_team'] = df['home_team'].replace(team_name_mapping)
# df['away_team'] = df['away_team'].replace(team_name_mapping)

# Ensure team names are strings to avoid sorting errors
df['home_team'] = df['home_team'].astype(str)
df['away_team'] = df['away_team'].astype(str)

# Create team encodings
teams = sorted(list(set(df['home_team'].unique()) | set(df['away_team'].unique())))
team_to_id = {team: i for i, team in enumerate(teams)}
df['home_team_id'] = df['home_team'].map(team_to_id)
df['away_team_id'] = df['away_team'].map(team_to_id)

print("Teams in dataset:", teams)
print("Total number of teams:", len(teams))
print("Bucknell matches:", len(df[(df['home_team'] == 'Bucknell') | (df['away_team'] == 'Bucknell')]))

# Sort by date to ensure chronological order for time series modeling
df = df.sort_values('Date')

# Create result column - from perspective of team1 vs team2
# W = team1 wins, L = team1 loses, D = draw
df['Result'] = np.where(df['goals_home'] > df['goals_away'], 'W',
                        np.where(df['goals_home'] < df['goals_away'], 'L', 'D'))

# Compute goal difference
df['goal_diff'] = df['goals_home'] - df['goals_away']
df['total_goals'] = df['goals_home'] + df['goals_away']
df['home_shot_conversion'] = df['goals_home'] / df['shots_home'].replace(0, 1)  # Avoid division by zero
df['away_shot_conversion'] = df['goals_away'] / df['shots_away'].replace(0, 1)  # Avoid division by zero

# Add recency weighting (more weight to recent matches)
max_date = df['Date'].max()
df['days_ago'] = (max_date - df['Date']).dt.days
df['recency_weight'] = 1 / (1 + df['days_ago']/365)  # Higher weight for recent matches

# Create team statistics based on previous matches
team_stats = {}

# Initialize team stats
for team in teams:
    team_stats[team] = {
        'matches_played': 0,
        'wins': 0,
        'draws': 0,
        'losses': 0,
        'goals_for': 0,
        'goals_against': 0,
        'shots': 0,
        'shots_on_target': 0,
        'corners': 0,
        'fouls': 0,
        'home_wins': 0,
        'home_draws': 0,
        'home_losses': 0,
        'away_wins': 0,
        'away_draws': 0,
        'away_losses': 0,
        'home_matches': 0,
        'away_matches': 0,
        'home_goals_for': 0,
        'home_goals_against': 0,
        'away_goals_for': 0,
        'away_goals_against': 0,
        'weighted_goals_for': 0,  # For recency weighting
        'weighted_goals_against': 0,  # For recency weighting
        'last_5_results': [],  # Store last 5 results for form
        # Add season-specific tracking
        'matches_2023': 0,
        'matches_2024': 0,
        'points_2023': 0,
        'points_2024': 0
    }

# Create head-to-head records
h2h_stats = {}
for team1 in teams:
    h2h_stats[team1] = {}
    for team2 in teams:
        if team1 != team2:
            h2h_stats[team1][team2] = {
                'matches': 0,
                'wins': 0,
                'draws': 0,
                'losses': 0,
                'goals_for': 0,
                'goals_against': 0,
                # Add season specific tracking
                'matches_2023': 0,
                'matches_2024': 0,
                'wins_2023': 0,
                'wins_2024': 0
            }

# Calculate team stats from match data
for _, row in df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    season = row['Season']
    recency = row.get('recency_weight', 1.0)  # Default to 1.0 if missing

    # Update head-to-head records
    if home_team != away_team:  # Skip if same team (shouldn't happen in real data)
        h2h_stats[home_team][away_team]['matches'] += 1
        h2h_stats[away_team][home_team]['matches'] += 1

        # Update season-specific h2h stats
        season_key = f'matches_{season}'
        if season_key in h2h_stats[home_team][away_team]:
            h2h_stats[home_team][away_team][season_key] += 1
            h2h_stats[away_team][home_team][season_key] += 1

        if row['Result'] == 'W':  # Home team wins
            h2h_stats[home_team][away_team]['wins'] += 1
            h2h_stats[away_team][home_team]['losses'] += 1

            # Season-specific win tracking
            win_key = f'wins_{season}'
            if win_key in h2h_stats[home_team][away_team]:
                h2h_stats[home_team][away_team][win_key] += 1

        elif row['Result'] == 'L':  # Away team wins
            h2h_stats[home_team][away_team]['losses'] += 1
            h2h_stats[away_team][home_team]['wins'] += 1

            # Season-specific win tracking
            win_key = f'wins_{season}'
            if win_key in h2h_stats[away_team][home_team]:
                h2h_stats[away_team][home_team][win_key] += 1

        else:  # Draw
            h2h_stats[home_team][away_team]['draws'] += 1
            h2h_stats[away_team][home_team]['draws'] += 1

        h2h_stats[home_team][away_team]['goals_for'] += row['goals_home']
        h2h_stats[home_team][away_team]['goals_against'] += row['goals_away']
        h2h_stats[away_team][home_team]['goals_for'] += row['goals_away']
        h2h_stats[away_team][home_team]['goals_against'] += row['goals_home']

    # Update home team stats
    team_stats[home_team]['matches_played'] += 1
    team_stats[home_team]['home_matches'] += 1

    # Track season-specific stats
    season_match_key = f'matches_{season}'
    if season_match_key in team_stats[home_team]:
        team_stats[home_team][season_match_key] += 1

    if row['Result'] == 'W':
        team_stats[home_team]['wins'] += 1
        team_stats[home_team]['home_wins'] += 1

        # Update season-specific points
        season_pts_key = f'points_{season}'
        if season_pts_key in team_stats[home_team]:
            team_stats[home_team][season_pts_key] += 3

    elif row['Result'] == 'D':
        team_stats[home_team]['draws'] += 1
        team_stats[home_team]['home_draws'] += 1

        # Update season-specific points
        season_pts_key = f'points_{season}'
        if season_pts_key in team_stats[home_team]:
            team_stats[home_team][season_pts_key] += 1

    else:
        team_stats[home_team]['losses'] += 1
        team_stats[home_team]['home_losses'] += 1

    team_stats[home_team]['goals_for'] += row['goals_home']
    team_stats[home_team]['goals_against'] += row['goals_away']
    team_stats[home_team]['home_goals_for'] += row['goals_home']
    team_stats[home_team]['home_goals_against'] += row['goals_away']

    # Apply recency weighting
    team_stats[home_team]['weighted_goals_for'] += row['goals_home'] * recency
    team_stats[home_team]['weighted_goals_against'] += row['goals_away'] * recency

    # Add columns if they exist
    for stat_col, team_stat in [
        ('shots_home', 'shots'),
        ('corners_home', 'corners'),
        ('fouls_home', 'fouls')
    ]:
        if stat_col in row and pd.notna(row[stat_col]):
            team_stats[home_team][team_stat] += row[stat_col]

    # Add result to last 5 results (from home team perspective)
    team_stats[home_team]['last_5_results'].append(row['Result'])
    if len(team_stats[home_team]['last_5_results']) > 5:
        team_stats[home_team]['last_5_results'] = team_stats[home_team]['last_5_results'][-5:]

    # Update away team stats
    team_stats[away_team]['matches_played'] += 1
    team_stats[away_team]['away_matches'] += 1

    # Track season-specific stats
    if season_match_key in team_stats[away_team]:
        team_stats[away_team][season_match_key] += 1

    if row['Result'] == 'L':
        team_stats[away_team]['wins'] += 1
        team_stats[away_team]['away_wins'] += 1

        # Update season-specific points
        season_pts_key = f'points_{season}'
        if season_pts_key in team_stats[away_team]:
            team_stats[away_team][season_pts_key] += 3

    elif row['Result'] == 'D':
        team_stats[away_team]['draws'] += 1
        team_stats[away_team]['away_draws'] += 1

        # Update season-specific points
        season_pts_key = f'points_{season}'
        if season_pts_key in team_stats[away_team]:
            team_stats[away_team][season_pts_key] += 1

    else:
        team_stats[away_team]['losses'] += 1
        team_stats[away_team]['away_losses'] += 1

    team_stats[away_team]['goals_for'] += row['goals_away']
    team_stats[away_team]['goals_against'] += row['goals_home']
    team_stats[away_team]['away_goals_for'] += row['goals_away']
    team_stats[away_team]['away_goals_against'] += row['goals_home']

    # Apply recency weighting
    team_stats[away_team]['weighted_goals_for'] += row['goals_away'] * recency
    team_stats[away_team]['weighted_goals_against'] += row['goals_home'] * recency

    # Add columns if they exist
    for stat_col, team_stat in [
        ('shots_away', 'shots'),
        ('corners_away', 'corners'),
        ('fouls_away', 'fouls')
    ]:
        if stat_col in row and pd.notna(row[stat_col]):
            team_stats[away_team][team_stat] += row[stat_col]

    # Add result to last 5 results (from away team perspective)
    away_result = 'W' if row['Result'] == 'L' else 'L' if row['Result'] == 'W' else 'D'
    team_stats[away_team]['last_5_results'].append(away_result)
    if len(team_stats[away_team]['last_5_results']) > 5:
        team_stats[away_team]['last_5_results'] = team_stats[away_team]['last_5_results'][-5:]

# Create calculated stats
for team in team_stats:
    stats = team_stats[team]
    matches = stats['matches_played']
    home_matches = max(stats['home_matches'], 1)  # Avoid division by zero
    away_matches = max(stats['away_matches'], 1)  # Avoid division by zero

    if matches > 0:
        stats['points'] = stats['wins'] * 3 + stats['draws']
        stats['ppg'] = stats['points'] / matches
        stats['win_rate'] = stats['wins'] / matches
        stats['draw_rate'] = stats['draws'] / matches
        stats['loss_rate'] = stats['losses'] / matches
        stats['goal_diff'] = stats['goals_for'] - stats['goals_against']
        stats['avg_goals_for'] = stats['goals_for'] / matches
        stats['avg_goals_against'] = stats['goals_against'] / matches
        stats['avg_shots'] = stats['shots'] / matches
        stats['avg_corners'] = stats['corners'] / matches
        stats['avg_fouls'] = stats['fouls'] / matches

        # Add recency-weighted stats
        stats['weighted_avg_goals_for'] = stats['weighted_goals_for'] / matches
        stats['weighted_avg_goals_against'] = stats['weighted_goals_against'] / matches

        # Home-away specific stats
        stats['home_ppg'] = (stats['home_wins'] * 3 + stats['home_draws']) / home_matches
        stats['away_ppg'] = (stats['away_wins'] * 3 + stats['away_draws']) / away_matches
        stats['home_win_rate'] = stats['home_wins'] / home_matches
        stats['away_win_rate'] = stats['away_wins'] / away_matches
        stats['home_avg_goals_for'] = stats['home_goals_for'] / home_matches
        stats['home_avg_goals_against'] = stats['home_goals_against'] / home_matches
        stats['away_avg_goals_for'] = stats['away_goals_for'] / away_matches
        stats['away_avg_goals_against'] = stats['away_goals_against'] / away_matches

        # Season-specific stats
        for season in [2023, 2024]:
            season_matches = stats.get(f'matches_{season}', 0)
            if season_matches > 0:
                stats[f'ppg_{season}'] = stats.get(f'points_{season}', 0) / season_matches
            else:
                stats[f'ppg_{season}'] = 0

        # Form calculation (points from last 5 games)
        recent_results = stats['last_5_results']
        form_points = sum([3 if res == 'W' else 1 if res == 'D' else 0 for res in recent_results])
        stats['form'] = form_points / (len(recent_results) * 3)  # As percentage of maximum possible
    else:
        # Default values for teams with no matches
        stats['points'] = 0
        stats['ppg'] = 0
        stats['win_rate'] = 0
        stats['draw_rate'] = 0
        stats['loss_rate'] = 0
        stats['goal_diff'] = 0
        stats['avg_goals_for'] = 0
        stats['avg_goals_against'] = 0
        stats['avg_shots'] = 0
        stats['avg_corners'] = 0
        stats['avg_fouls'] = 0
        stats['home_ppg'] = 0
        stats['away_ppg'] = 0
        stats['home_win_rate'] = 0
        stats['away_win_rate'] = 0
        stats['home_avg_goals_for'] = 0
        stats['home_avg_goals_against'] = 0
        stats['away_avg_goals_for'] = 0
        stats['away_avg_goals_against'] = 0
        stats['form'] = 0.5  # Neutral form for teams with no data
        stats['weighted_avg_goals_for'] = 0
        stats['weighted_avg_goals_against'] = 0

        # Season-specific defaults
        stats['ppg_2023'] = 0
        stats['ppg_2024'] = 0

# Create team statistics dataframe
team_stats_df = pd.DataFrame([
    {
        'Team': team,
        'Matches': stats['matches_played'],
        'Wins': stats['wins'],
        'Draws': stats['draws'],
        'Losses': stats['losses'],
        'GF': stats['goals_for'],
        'GA': stats['goals_against'],
        'GD': stats['goal_diff'],
        'Points': stats['points'],
        'PPG': stats['ppg'],
        'Win%': stats['win_rate'] * 100,
        'Home PPG': stats['home_ppg'],
        'Away PPG': stats['away_ppg'],
        'Form': stats['form'] * 100,
        # Add season specific stats
        'Matches 2023': stats.get('matches_2023', 0),
        'PPG 2023': stats.get('ppg_2023', 0),
        'Matches 2024': stats.get('matches_2024', 0),
        'PPG 2024': stats.get('ppg_2024', 0)
    }
    for team, stats in team_stats.items()
]).sort_values('Points', ascending=False)

print("\nTeam Performance Table:")
print(team_stats_df[['Team', 'Matches', 'Wins', 'Draws', 'Losses', 'Points', 'PPG', 'GF', 'GA', 'GD', 'Form']])

# Enhanced feature engineering for match prediction
def create_feature_vector(row):
    home_team = row['home_team']
    away_team = row['away_team']

    home_stats = team_stats[home_team]
    away_stats = team_stats[away_team]

    # Get head-to-head stats if available
    h2h = h2h_stats[home_team][away_team] if home_team != away_team and away_team in h2h_stats.get(home_team, {}) else {
        'matches': 0, 'wins': 0, 'draws': 0, 'losses': 0, 'goals_for': 0, 'goals_against': 0
    }

    # Calculate head-to-head strength (how well home team performs against away team)
    if h2h.get('matches', 0) > 0:
        h2h_win_rate = h2h['wins'] / h2h['matches']
        h2h_ppg = (h2h['wins'] * 3 + h2h['draws']) / h2h['matches']
        h2h_goal_diff = (h2h['goals_for'] - h2h['goals_against']) / h2h['matches']

        # Recent h2h performance (last season vs all time)
        matches_2024 = h2h.get('matches_2024', 0)
        if matches_2024 > 0:
            h2h_win_rate_2024 = h2h.get('wins_2024', 0) / matches_2024
        else:
            h2h_win_rate_2024 = h2h_win_rate  # Fall back to overall if no recent data
    else:
        # Default values if no h2h history
        h2h_win_rate = 0.5
        h2h_ppg = home_stats['ppg'] - away_stats['ppg'] + 0.5  # Default to PPG difference plus home advantage
        h2h_goal_diff = 0
        h2h_win_rate_2024 = 0.5

    # Create a comprehensive feature vector with enhanced stats
    features = [
        # Basic team IDs
        row['home_team_id'],
        row['away_team_id'],

        # Overall team performance
        home_stats['ppg'],
        away_stats['ppg'],
        home_stats['win_rate'],
        away_stats['win_rate'],
        home_stats['draw_rate'],
        away_stats['draw_rate'],

        # Goal stats
        home_stats['avg_goals_for'],
        home_stats['avg_goals_against'],
        away_stats['avg_goals_for'],
        away_stats['avg_goals_against'],

        # Recency-weighted goal stats
        home_stats.get('weighted_avg_goals_for', home_stats['avg_goals_for']),
        home_stats.get('weighted_avg_goals_against', home_stats['avg_goals_against']),
        away_stats.get('weighted_avg_goals_for', away_stats['avg_goals_for']),
        away_stats.get('weighted_avg_goals_against', away_stats['avg_goals_against']),

        # Strength at home/away
        home_stats['home_ppg'],
        away_stats['away_ppg'],
        home_stats['home_win_rate'],
        away_stats['away_win_rate'],
        home_stats['home_avg_goals_for'],
        home_stats['home_avg_goals_against'],
        away_stats['away_avg_goals_for'],
        away_stats['away_avg_goals_against'],

        # Recent form
        home_stats['form'],
        away_stats['form'],

        # Season-specific performance
        home_stats.get('ppg_2023', 0),
        home_stats.get('ppg_2024', 0),
        away_stats.get('ppg_2023', 0),
        away_stats.get('ppg_2024', 0),

        # Head-to-head stats
        h2h_win_rate,
        h2h_ppg,
        h2h_goal_diff,
        h2h_win_rate_2024,  # Recent h2h performance

        # Game stats
        home_stats['avg_shots'],
        away_stats['avg_shots'],
        home_stats['avg_corners'],
        away_stats['avg_corners'],

        # Strategic insights
        home_stats['avg_goals_for'] - away_stats['avg_goals_against'],  # Home attack vs Away defense
        away_stats['avg_goals_for'] - home_stats['avg_goals_against'],  # Away attack vs Home defense

        # Momentum (goal difference in last few games)
        home_stats['goal_diff'] / max(home_stats['matches_played'], 1),
        away_stats['goal_diff'] / max(away_stats['matches_played'], 1)
    ]

    return features

# Function to create a simplified model equation for Excel
def create_excel_equation(model, feature_names, output_class=0):
    """
    Create a simplified equation for Excel based on feature importances.
    For random forest, we use the feature importances as coefficients.

    Parameters:
    model: Trained model with feature_importances_ attribute
    feature_names: List of feature names
    output_class: Class index for multi-class problems (0=home win, 1=draw, 2=away win)

    Returns:
    Excel formula string and coefficient dictionary
    """
    if not hasattr(model, 'feature_importances_'):
        print("Model does not have feature importances attribute")
        return None, None

    # Get feature importances
    importances = model.feature_importances_

    # Create coefficients dictionary
    coefficients = {feature: importance for feature, importance in zip(feature_names, importances)}

    # Create Excel formula string
    excel_formula = "="
    excel_terms = []

    for feature, coef in coefficients.items():
        # Skip team IDs as they're categorical
        if 'Team ID' in feature:
            continue

        # Format coefficient to 4 decimal places
        coef_str = f"{coef:.4f}"

        # Add the term: coefficient * feature
        excel_terms.append(f"{coef_str}*{feature}")

    # Join terms with +
    excel_formula += "+".join(excel_terms)

    # Return the formula and coefficients
    return excel_formula, coefficients

# Class for custom weighted predictions
class CustomWeightedPredictor:
    def __init__(self, base_model, feature_names, custom_weights=None):
        self.base_model = base_model
        self.feature_names = feature_names

        # Default weights if none provided
        if custom_weights is None:
            self.custom_weights = {
                'H2H Home PPG': 0.25,
                'H2H Goal Diff per Game': 0.20,
                'H2H Home Win Rate': 0.15,
                'Away Attack vs Home Defense': 0.08,
                'Home Draw Rate': 0.02,
                'Home PPG': 0.04,
                'Away PPG': 0.04,
                'Home Recent Form': 0.06,
                'Away Recent Form': 0.06,
                'Home Avg Goals For': 0.05,
                'Away Avg Goals For': 0.05
            }
        else:
            self.custom_weights = custom_weights

        # Create weight array
        self.weight_array = np.zeros(len(feature_names))
        for feature, weight in self.custom_weights.items():
            if feature in feature_names:
                idx = feature_names.index(feature)
                self.weight_array[idx] = weight
            else:
                print(f"Warning: Feature '{feature}' not found in model")

        # Normalize weights
        if np.sum(self.weight_array) > 0:
            self.weight_array = self.weight_array / np.sum(self.weight_array)

        # Print the custom weights
        print("\nCustomized Feature Importance:")
        feature_importance = sorted(zip(feature_names, self.weight_array),
                                    key=lambda x: x[1], reverse=True)
        for i, (feature, importance) in enumerate(feature_importance[:15]):
            print(f"{i+1}. {feature}: {importance:.4f}")

    def predict(self, X):
        """
        Makes predictions using the base model
        """
        return self.base_model.predict(X)

    def predict_proba(self, X):
        """
        Makes probability predictions using the base model
        """
        return self.base_model.predict_proba(X)

    def get_excel_formula(self, output_class=0):
        """
        Create Excel formula using custom weights
        """
        # Create Excel formula string
        excel_formula = "="
        excel_terms = []

        # Sort weights by importance
        sorted_weights = sorted(zip(self.feature_names, self.weight_array),
                                key=lambda x: x[1], reverse=True)

        for feature, weight in sorted_weights:
            # Skip team IDs as they're categorical
            if 'Team ID' in feature:
                continue

            # Format weight to 4 decimal places
            weight_str = f"{weight:.4f}"

            # Add the term: weight * feature
            if weight > 0.001:  # Only include non-zero weights
                excel_terms.append(f"{weight_str}*{feature}")

        # Join terms with +
        excel_formula += "+".join(excel_terms)

        return excel_formula

# Create a simplified logistic regression model for interpretability
def create_logistic_regression_model(X, y, feature_names):
    """
    Create a logistic regression model that's easily interpretable

    Parameters:
    X: Feature matrix
    y: Target vector
    feature_names: List of feature names

    Returns:
    Trained logistic regression model
    """
    # Scale features for better convergence
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train a logistic regression model
    lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, C=1.0)
    lr_model.fit(X_scaled, y)

    # Print coefficients
    print("\nLogistic Regression Coefficients (for easier interpretation):")
    for i, class_name in enumerate(['Home Win', 'Draw', 'Away Win']):
        print(f"\nFor {class_name}:")
        coeffs = sorted(zip(feature_names, lr_model.coef_[i]), key=lambda x: abs(x[1]), reverse=True)
        for feature, coeff in coeffs[:10]:  # Top 10 coefficients
            print(f"  {feature}: {coeff:.4f}")

    # Create Excel formula from logistic regression
    formulas = {}
    for i, class_name in enumerate(['Home Win', 'Draw', 'Away Win']):
        formula = "="
        terms = []
        # Add intercept
        terms.append(f"{lr_model.intercept_[i]:.4f}")

        for feature, coeff in zip(feature_names, lr_model.coef_[i]):
            # Skip team IDs or zero/near-zero coefficients
            if 'Team ID' in feature or abs(coeff) < 0.01:
                continue

            # Format coefficient
            coeff_str = f"{coeff:.4f}"

            # Add the term: coefficient * feature
            terms.append(f"{coeff_str}*{feature}")

        # Join terms with +
        formula += "+".join(terms)
        formulas[class_name] = formula

    return lr_model, scaler, formulas

# Prepare train/test data
X = np.array([create_feature_vector(row) for _, row in df.iterrows()])
y = np.array([0 if r == 'W' else 1 if r == 'D' else 2 for r in df['Result']])

# List of feature names (for importance analysis)
feature_names = [
    'Home Team ID', 'Away Team ID',
    'Home PPG', 'Away PPG',
    'Home Win Rate', 'Away Win Rate',
    'Home Draw Rate', 'Away Draw Rate',
    'Home Avg Goals For', 'Home Avg Goals Against',
    'Away Avg Goals For', 'Away Avg Goals Against',
    'Home Weighted Avg Goals For', 'Home Weighted Avg Goals Against',
    'Away Weighted Avg Goals For', 'Away Weighted Avg Goals Against',
    'Home PPG at Home', 'Away PPG Away',
    'Home Win Rate at Home', 'Away Win Rate Away',
    'Home Avg Goals For at Home', 'Home Avg Goals Against at Home',
    'Away Avg Goals For Away', 'Away Avg Goals Against Away',
    'Home Recent Form', 'Away Recent Form',
    'Home PPG 2023', 'Home PPG 2024',
    'Away PPG 2023', 'Away PPG 2024',
    'H2H Home Win Rate', 'H2H Home PPG', 'H2H Goal Diff per Game', 'H2H Home Win Rate 2024',
    'Home Avg Shots', 'Away Avg Shots',
    'Home Avg Corners', 'Away Avg Corners',
    'Home Attack vs Away Defense', 'Away Attack vs Home Defense',
    'Home Goal Diff per Game', 'Away Goal Diff per Game'
]

# Add this before model training
from sklearn.model_selection import train_test_split

# Create chronological train/test split (last 20% of matches as test)
split_idx = int(len(df) * 0.8)
train_indices = df.index[:split_idx]
test_indices = df.index[split_idx:]

X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]

# Train models on training data only
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)

# Evaluate on test data
rf_test_acc = accuracy_score(y_test, rf_model.predict(X_test))
gb_test_acc = accuracy_score(y_test, gb_model.predict(X_test))
print(f"Random Forest Test accuracy: {rf_test_acc:.4f}")
print(f"Gradient Boosting Test accuracy: {gb_test_acc:.4f}")

# Try both Random Forest and Gradient Boosting
print("\nTraining models with enhanced features...")

# Train a Random Forest classifier with tuned parameters
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42
)
rf_model.fit(X, y)

# Train a Gradient Boosting classifier
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
gb_model.fit(X, y)

# Evaluate the models with cross-validation
# Use TimeSeriesSplit for chronological data
tscv = TimeSeriesSplit(n_splits=5)
rf_cv_scores = cross_val_score(rf_model, X, y, cv=tscv)
gb_cv_scores = cross_val_score(gb_model, X, y, cv=tscv)

print(f"\nRandom Forest Cross-validation accuracy: {np.mean(rf_cv_scores):.4f}")
print(f"Gradient Boosting Cross-validation accuracy: {np.mean(gb_cv_scores):.4f}")

# Use the better performing model
if np.mean(rf_cv_scores) >= np.mean(gb_cv_scores):
    model = rf_model
    print("Using Random Forest model for predictions (better performance)")
else:
    model = gb_model
    print("Using Gradient Boosting model for predictions (better performance)")

# Feature importance before customization
if hasattr(model, 'feature_importances_'):
    print("\nOriginal Feature Importance:")
    feature_importance = sorted(zip(feature_names, model.feature_importances_),
                                key=lambda x: x[1], reverse=True)
    for i, (feature, importance) in enumerate(feature_importance[:15]):
        print(f"{i+1}. {feature}: {importance:.4f}")

# Evaluate model on Bucknell matches
bucknell_indices = df[(df['home_team'] == 'Bucknell') | (df['away_team'] == 'Bucknell')].index
X_bucknell = X[bucknell_indices]
y_bucknell = y[bucknell_indices]
bucknell_matches = df.iloc[bucknell_indices].reset_index(drop=True)

# Get model predictions
y_pred = model.predict(X_bucknell)
y_proba = model.predict_proba(X_bucknell)

# Calculate accuracy for Bucknell matches
bucknell_accuracy = accuracy_score(y_bucknell, y_pred)
print(f"\nModel accuracy on Bucknell matches: {bucknell_accuracy:.4f}")

# Create a DataFrame to show results
bucknell_results = pd.DataFrame({
    'Date': bucknell_matches['Date'],
    'Home Team': bucknell_matches['home_team'],
    'Away Team': bucknell_matches['away_team'],
    'Actual Result': bucknell_matches['Result'],
    'Predicted Result': ['W' if p == 0 else 'D' if p == 1 else 'L' for p in y_pred],
    'Win Prob': [round(p[0] * 100, 1) for p in y_proba],
    'Draw Prob': [round(p[1] * 100, 1) for p in y_proba],
    'Loss Prob': [round(p[2] * 100, 1) for p in y_proba]
})

# Determine if prediction was correct
bucknell_results['Correct'] = bucknell_results['Actual Result'] == bucknell_results['Predicted Result']

# Print detailed results
print("\nBucknell Match Results with Base Model:")
print(bucknell_results[['Date', 'Home Team', 'Away Team', 'Actual Result', 'Predicted Result', 'Correct']])

# Print classification report for overall model
print("\nOverall Model Performance:")
print(classification_report(y, model.predict(X)))

# Print overall accuracy
overall_accuracy = accuracy_score(y, model.predict(X))
print(f"Overall model accuracy: {overall_accuracy:.4f}")

Total matches after filtering team '0': 181
Teams after standardization: ['American', 'Army West Point', 'BU', 'Bucknell', 'Colgate', 'Holy Cross', 'Lafayette', 'Lehigh', 'Loyola', 'Navy']
Total unique teams after standardization: 10
Teams in dataset: ['American', 'Army West Point', 'BU', 'Bucknell', 'Colgate', 'Holy Cross', 'Lafayette', 'Lehigh', 'Loyola', 'Navy']
Total number of teams: 10
Bucknell matches: 37

Team Performance Table:
              Team  Matches  Wins  Draws  Losses  Points       PPG  GF  GA  \
3         Bucknell       37    24      6       7      78  2.108108  50  23   
1  Army West Point       36    22      4      10      70  1.944444  54  24   
2               BU       37    19     12       6      69  1.864865  77  42   
8           Loyola       36    18      6      12      60  1.666667  54  32   
4          Colgate       36    12      8      16      44  1.222222  42  50   
7           Lehigh       36    12      4      20      40  1.111111  38  60   
9             

In [9]:

# Train a logistic regression model for interpretability
lr_model, scaler, lr_formulas = create_logistic_regression_model(X, y, feature_names)

# Custom weights - modify these values as needed
custom_weights = {
    'H2H Home PPG': 0.22,                   # Slightly decreased
    'H2H Goal Diff per Game': 0.18,         # Slightly decreased
    'H2H Home Win Rate': 0.14,              # Slightly decreased
    'H2H Home Win Rate 2024': 0.08,         # New feature for recent h2h performance
    'Away Attack vs Home Defense': 0.07,    # Slightly decreased
    'Home Draw Rate': 0.02,                 # Same
    'Home PPG': 0.03,                       # Slightly decreased
    'Away PPG': 0.03,                       # Slightly decreased
    'Home Recent Form': 0.05,               # Slightly decreased
    'Away Recent Form': 0.05,               # Slightly decreased
    'Home Avg Goals For': 0.04,             # Slightly decreased
    'Away Avg Goals For': 0.04,             # Slightly decreased
    'Home PPG 2024': 0.03,                  # New feature for recent season
    'Away PPG 2024': 0.02,                  # New feature for recent season
}



# Create a custom weighted predictor
custom_model = CustomWeightedPredictor(model, feature_names, custom_weights)

# Generate Excel formulas
rf_formula, rf_coefs = create_excel_equation(model, feature_names, output_class=0)
custom_formula = custom_model.get_excel_formula(output_class=0)

print("\n==== EXCEL FORMULAS ====")
print("\n1. Random Forest Model Formula (Home Win Probability):")
print(rf_formula)

print("\n2. Custom Weighted Formula (Based on your custom weights):")
print(custom_formula)

print("\n3. Logistic Regression Formulas (more suitable for Excel):")
for outcome, formula in lr_formulas.items():
    print(f"\nFor {outcome}:")
    print(formula)

#Export to excel

# Create a comprehensive team stats dataframe for export
detailed_team_stats = []

for team in teams:
    if team == '0':  # Skip the '0' team if it exists
        continue

    stats = team_stats[team]

    # Create a dictionary with all stats for this team
    team_data = {
        'Team': team,
        # Basic stats
        'Matches_Played': stats['matches_played'],
        'Wins': stats['wins'],
        'Draws': stats['draws'],
        'Losses': stats['losses'],
        'Goals_For': stats['goals_for'],
        'Goals_Against': stats['goals_against'],
        'Goal_Diff': stats.get('goal_diff', 0),
        'Points': stats.get('points', 0),
        'PPG': stats.get('ppg', 0),

        # Home/Away breakdown
        'Home_Matches': stats['home_matches'],
        'Away_Matches': stats['away_matches'],
        'Home_Wins': stats['home_wins'],
        'Home_Draws': stats['home_draws'],
        'Home_Losses': stats['home_losses'],
        'Away_Wins': stats['away_wins'],
        'Away_Draws': stats['away_draws'],
        'Away_Losses': stats['away_losses'],

        # Advanced stats
        'Win_Rate': stats.get('win_rate', 0) * 100,  # Convert to percentage
        'Draw_Rate': stats.get('draw_rate', 0) * 100,
        'Loss_Rate': stats.get('loss_rate', 0) * 100,
        'Home_Win_Rate': stats.get('home_win_rate', 0) * 100,
        'Away_Win_Rate': stats.get('away_win_rate', 0) * 100,
        'Home_PPG': stats.get('home_ppg', 0),
        'Away_PPG': stats.get('away_ppg', 0),

        # Offensive/Defensive metrics
        'Avg_Goals_For': stats.get('avg_goals_for', 0),
        'Avg_Goals_Against': stats.get('avg_goals_against', 0),
        'Home_Avg_Goals_For': stats.get('home_avg_goals_for', 0),
        'Home_Avg_Goals_Against': stats.get('home_avg_goals_against', 0),
        'Away_Avg_Goals_For': stats.get('away_avg_goals_for', 0),
        'Away_Avg_Goals_Against': stats.get('away_avg_goals_against', 0),

        # Game stats
        'Avg_Shots': stats.get('avg_shots', 0),
        'Avg_Corners': stats.get('avg_corners', 0),
        'Avg_Fouls': stats.get('avg_fouls', 0),

        # Season-specific stats
        'Matches_2023': stats.get('matches_2023', 0),
        'PPG_2023': stats.get('ppg_2023', 0),
        'Matches_2024': stats.get('matches_2024', 0),
        'PPG_2024': stats.get('ppg_2024', 0),

        # Form
        'Form': stats.get('form', 0) * 100  # Convert to percentage
    }

    # Add head-to-head stats against each opponent
    for opponent in teams:
        if opponent == '0' or opponent == team:  # Skip the '0' team and self matchups
            continue

        if opponent in h2h_stats.get(team, {}):
            h2h = h2h_stats[team][opponent]

            # Calculate H2H metrics if there are matches
            if h2h['matches'] > 0:
                h2h_win_rate = h2h['wins'] / h2h['matches'] * 100  # As percentage
                h2h_ppg = (h2h['wins'] * 3 + h2h['draws']) / h2h['matches']
                h2h_goal_diff = (h2h['goals_for'] - h2h['goals_against']) / h2h['matches']
            else:
                h2h_win_rate = 0
                h2h_ppg = 0
                h2h_goal_diff = 0

            # Add H2H stats with opponent name in column
            team_data[f'H2H_{opponent}_Matches'] = h2h['matches']
            team_data[f'H2H_{opponent}_Wins'] = h2h['wins']
            team_data[f'H2H_{opponent}_Draws'] = h2h['draws']
            team_data[f'H2H_{opponent}_Losses'] = h2h['losses']
            team_data[f'H2H_{opponent}_Goals_For'] = h2h['goals_for']
            team_data[f'H2H_{opponent}_Goals_Against'] = h2h['goals_against']
            team_data[f'H2H_{opponent}_Win_Rate'] = h2h_win_rate
            team_data[f'H2H_{opponent}_PPG'] = h2h_ppg
            team_data[f'H2H_{opponent}_Goal_Diff_Per_Game'] = h2h_goal_diff

            # Add season-specific H2H stats
            for season in [2023, 2024]:
                matches_key = f'matches_{season}'
                wins_key = f'wins_{season}'
                season_matches = h2h.get(matches_key, 0)
                season_wins = h2h.get(wins_key, 0)

                team_data[f'H2H_{opponent}_Matches_{season}'] = season_matches
                team_data[f'H2H_{opponent}_Wins_{season}'] = season_wins

                if season_matches > 0:
                    team_data[f'H2H_{opponent}_Win_Rate_{season}'] = season_wins / season_matches * 100
                else:
                    team_data[f'H2H_{opponent}_Win_Rate_{season}'] = 0

    detailed_team_stats.append(team_data)

# Create DataFrame from collected data
detailed_stats_df = pd.DataFrame(detailed_team_stats)

# Export to Excel
excel_filename = 'patriot_league_team_stats_combined.xlsx'
detailed_stats_df.to_excel(excel_filename, index=False)
print(f"\nDetailed team statistics exported to {excel_filename}")

# Make predictions for Bucknell against all other teams
bucknell_predictions = []

# Get all other teams
other_teams = [team for team in teams if team != 'Bucknell' and team != '0']

# For each opponent, predict Bucknell as both home and away
for opponent in other_teams:
    # Bucknell as home team
    home_features = create_feature_vector({
        'home_team': 'Bucknell',
        'away_team': opponent,
        'home_team_id': team_to_id['Bucknell'],
        'away_team_id': team_to_id[opponent]
    })

    # Bucknell as away team
    away_features = create_feature_vector({
        'home_team': opponent,
        'away_team': 'Bucknell',
        'home_team_id': team_to_id[opponent],
        'away_team_id': team_to_id['Bucknell']
    })

    # Predict both scenarios using custom model
    home_pred = custom_model.predict([home_features])[0]
    away_pred = custom_model.predict([away_features])[0]

    # Get probability estimates
    home_probs = custom_model.predict_proba([home_features])[0]
    away_probs = custom_model.predict_proba([away_features])[0]

    # Convert to match results from Bucknell's perspective
    bucknell_home_result = 'Win' if home_pred == 0 else 'Draw' if home_pred == 1 else 'Loss'
    bucknell_away_result = 'Win' if away_pred == 2 else 'Draw' if away_pred == 1 else 'Loss'

    # Add predictions
    bucknell_predictions.append({
        'Opponent': opponent,
        'Bucknell at Home': bucknell_home_result,
        'Win Prob (Home)': round(home_probs[0] * 100, 1),
        'Draw Prob (Home)': round(home_probs[1] * 100, 1),
        'Loss Prob (Home)': round(home_probs[2] * 100, 1),
        'Bucknell Away': bucknell_away_result,
        'Win Prob (Away)': round(away_probs[2] * 100, 1),
        'Draw Prob (Away)': round(away_probs[1] * 100, 1),
        'Loss Prob (Away)': round(away_probs[0] * 100, 1)
    })

# Create predictions dataframe
bucknell_pred_df = pd.DataFrame(bucknell_predictions)

# Display Bucknell predictions
print("\nBucknell Match Predictions with Customized Weights:")
print(bucknell_pred_df[['Opponent', 'Bucknell at Home', 'Bucknell Away']])

# Calculate expected points
home_points = sum([3 if r == 'Win' else 1 if r == 'Draw' else 0 for r in bucknell_pred_df['Bucknell at Home']])
away_points = sum([3 if r == 'Win' else 1 if r == 'Draw' else 0 for r in bucknell_pred_df['Bucknell Away']])
total_points = home_points + away_points
matches = len(bucknell_pred_df) * 2

print(f"\nExpected Points for Bucknell with Customized Model:")
print(f"Home Points: {home_points} from {len(bucknell_pred_df)} matches")
print(f"Away Points: {away_points} from {len(bucknell_pred_df)} matches")
print(f"Total Points: {total_points} from {matches} matches")
print(f"Expected PPG: {total_points / matches:.2f}")

# Detailed prediction breakdown
print("\nDetailed Bucknell Predictions with Customized Weights:")
for _, row in bucknell_pred_df.iterrows():
    print(f"vs {row['Opponent']}:")
    print(f"  At Home: {row['Bucknell at Home']} (Win: {row['Win Prob (Home)']}%, Draw: {row['Draw Prob (Home)']}%, Loss: {row['Loss Prob (Home)']}%)")
    print(f"  Away: {row['Bucknell Away']} (Win: {row['Win Prob (Away)']}%, Draw: {row['Draw Prob (Away)']}%, Loss: {row['Loss Prob (Away)']}%)")

# Summary of improvements and combined data benefits
print("\nMODEL IMPROVEMENTS SUMMARY:")
print("1. Team name standardization: Fixed type errors with team names")
print("2. Enhanced features:")
print("   - Home/away performance differentials")
print("   - Recent team form metrics")
print("   - Head-to-head statistics")
print("   - Attack vs defense matchup indicators")
print("   - Season-specific performance metrics (2023 vs 2024)")
print("   - Recency-weighted statistics")
print("3. Advanced modeling:")
print("   - Tuned Random Forest parameters")
print("   - Tested Gradient Boosting classifier")
print("   - Used time-series cross-validation")
print("4. Custom feature weights:")
print("   - Increased importance of head-to-head metrics")
print("   - Added emphasis on recent form")
print("   - Balanced attack/defense considerations")
print("   - Added weight to recent season performance")
print("5. Exportable equations:")
print("   - Random Forest feature importance")
print("   - Custom weighted formula")
print("   - Logistic Regression equations for Excel")
print("6. Combined data benefits:")
print("   - More data points for model training")
print("   - Ability to capture team performance trends across seasons")
print("   - Season-specific metrics provide temporal context")
print("   - Recency weighting gives more importance to recent performance")


Logistic Regression Coefficients (for easier interpretation):

For Home Win:
  H2H Home Win Rate: 1.9369
  Away Avg Goals For Away: -0.6375
  H2H Home PPG: 0.5182
  Away PPG Away: -0.4339
  Away PPG 2023: 0.4224
  Away Avg Goals Against Away: 0.3618
  Home Team ID: 0.3390
  Home PPG 2023: -0.2879
  Home PPG at Home: 0.2647
  Home Draw Rate: -0.2560

For Draw:
  H2H Home Win Rate: -2.7885
  H2H Home PPG: 1.8332
  H2H Goal Diff per Game: 0.6070
  Away PPG 2023: -0.4795
  Home Recent Form: -0.4793
  Home Team ID: -0.4448
  Home PPG at Home: 0.4196
  Home PPG 2023: 0.4119
  Home Weighted Avg Goals Against: 0.3715
  Home Avg Goals For at Home: 0.3715

For Away Win:
  H2H Home PPG: -2.3514
  H2H Home Win Rate: 0.8515
  H2H Goal Diff per Game: -0.8204
  Home PPG at Home: -0.6843
  Home Avg Goals For at Home: -0.5130
  Home Recent Form: 0.5075
  Away PPG Away: 0.4170
  Home Weighted Avg Goals Against: -0.4063
  Away PPG 2024: -0.3992
  Home Avg Goals Against at Home: 0.3825

Customized Featur